## Evaluating multi-group promps

In [1]:
import pandas as pd
import numpy as np
from labelrepo.projects.participant_demographics import get_participant_demographics
from labelrepo.database import get_database_connection

subgroups = get_participant_demographics(include_locations=True)
docs_info = pd.read_sql(
    "select pmcid, publication_year, title, text from document",
    get_database_connection(),

)

In [2]:
# Load multi group as well
jerome_pd = subgroups[(subgroups.project_name == 'participant_demographics') & \
                      (subgroups.annotator_name == 'Jerome_Dockes')]

counts = jerome_pd.groupby('pmcid').count().reset_index()
multi_group_pmcids = counts[counts['count'] > 1].pmcid
multi_group = jerome_pd[jerome_pd.pmcid.isin(multi_group_pmcids)]

# Get multi group docs
multi_group_docs = docs_info[docs_info.pmcid.isin(multi_group.pmcid)]

## Embed documents

In [3]:
import openai
openai.api_key = open('/home/zorro/.keys/open_ai.key').read().strip()

In [4]:
from embed import embed_pmc_articles, query_embeddings
texts = multi_group_docs[['pmcid', 'text']].to_dict(orient='records')
# embeddings = embed_pmc_articles(texts)
import pickle
# pickle.dump(embeddings, open('data/multi_group_embeddings.pkl', 'wb'))
embeddings = pickle.load(open('data/multi_group_embeddings.pkl', 'rb'))

In [5]:
embeddings = pd.DataFrame(embeddings)

## Extract group demographics across all docs 

In [6]:
from extract import extract_on_match
from templates import ZERO_SHOT_MULTI_GROUP
# sections, predictions = extract_on_match(embeddings, multi_group, **ZERO_SHOT_MULTI_GROUP, num_workers=3)
# pickle.dump(predictions, open('data/full_text_gpt4_multi_group.pkl', 'wb'))
predictions = pickle.load(open('data/full_text_gpt4_multi_group.pkl', 'rb'))

# Evaluation

In [7]:
# Subset annotation df to only include studies with body annotations
subset_cols = ['count', 'diagnosis', 'group_name', 'subgroup_name', 'male count',
       'female count', 'age mean', 'age minimum', 'age maximum',
       'age median', 'pmcid']
sub_multi_group = multi_group[multi_group.pmcid.isin(predictions.pmcid)][subset_cols].sort_values('pmcid')

In [8]:
from evaluate import evaluate_predictions, clean_predictions

In [9]:
clean_preds = clean_predictions(predictions)

In [10]:
ix_corr_n_groups, ix_more_groups, ix_less_groups = evaluate_predictions(predictions, sub_multi_group)

Exact match # of groups: 0.86
 More groups predicted: 0.09
 Less groups predicted: 0.06

Column wise comparison of predictions and annotations:

{'age maximum': 0.63,
 'age mean': 0.75,
 'age median': 0.77,
 'age minimum': 0.63,
 'count': 0.8,
 'diagnosis': 0.08,
 'female count': 0.65,
 'group_name': 0.65,
 'male count': 0.62,
 'subgroup_name': 0.03}


In [11]:
# With "cleaned up" predictions

_ = evaluate_predictions(clean_preds, sub_multi_group)

Exact match # of groups: 0.88
 More groups predicted: 0.06
 Less groups predicted: 0.06

Column wise comparison of predictions and annotations:

{'age maximum': 0.71,
 'age mean': 0.74,
 'age median': 0.85,
 'age minimum': 0.71,
 'count': 0.8,
 'diagnosis': 0.52,
 'female count': 0.75,
 'group_name': 0.79,
 'male count': 0.67,
 'subgroup_name': 0.03}


### Try on single group

In [12]:
single_group_embeddings = pickle.load(open('data/single_group_embeddings.pkl', 'rb'))
single_group_embeddings = pd.DataFrame(single_group_embeddings)

single_group_pmcids = counts[counts['count'] == 1].pmcid
single_group = jerome_pd[jerome_pd.pmcid.isin(single_group_pmcids)]

In [13]:
single_group_embeddings

,content,start_char,end_char,section_0,section_1,section_2,pmcid,embedding
0,"Pulli, Elmo P. and Silver, Eero and Kumpulaine...",0,323,NaN,NaN,NaN,9108497,"[-0.005361288785934448, -0.015115763060748577,..."
1,\n# Title\n\nFeasibility of FreeSurfer Process...,323,468,Title,NaN,NaN,9108497,"[-0.011751627549529076, 0.026484699919819832, ..."
2,\n# Keywords\n\nbrain\nchild\nneuroimaging\nbr...,468,563,Keywords,NaN,NaN,9108497,"[-0.02721841260790825, 0.02183225192129612, 0...."
3,\n# Abstract\n \nPediatric neuroimaging is a q...,563,2381,Abstract,NaN,NaN,9108497,"[-0.0069894008338451385, 0.03050420992076397, ..."
4,\n# Body\n \n## Introduction \n \nThere are m...,2381,5253,Body,Introduction,NaN,9108497,"[-0.0003534696879796684, 0.024464335292577744,..."
...,...,...,...,...,...,...,...,...
1726,\n## Data Availability Statement \n \nThe raw...,31764,31956,Body,Data Availability Statement,NaN,6908505,"[-0.016607334837317467, -0.02181965485215187, ..."
1727,\n## Ethics Statement \n \nThe studies involv...,31956,32200,Body,Ethics Statement,NaN,6908505,"[-0.008219190873205662, -0.012068435549736023,..."
1728,\n## Author Contributions \n \nKM designed th...,32200,32513,Body,Author Contributions,NaN,6908505,"[-0.0033632966224104166, 0.008040985092520714,..."
1729,\n## Funding \n \nThe study was based on two ...,32513,32823,Body,Funding,NaN,6908505,"[0.0009335727663710713, -0.024999098852276802,..."


In [14]:
single_sections, single_preds = extract_on_match(single_group_embeddings, single_group, **ZERO_SHOT_MULTI_GROUP, num_workers=3)
# single_preds = pickle.load(open('data/full_text_gpt4_single_group.pkl', 'rb'))

100%|███████████████████████████████████████████| 41/41 [00:36<00:00,  1.12it/s]


In [21]:
sub_single_group = single_group[single_group.pmcid.isin(single_preds.pmcid)][subset_cols].sort_values('pmcid')

In [24]:
single_preds

,count,is_matching_chunk,pmcid,content,annot_count,correct
0,121.0,True,9108497,\n### Participants \n \nThe participants are ...,134,False
1,216.0,True,8828908,\n## Materials and Methods \n \n### Participa...,216,True
2,112.0,True,8752963,\n## Methods \n \n### MEG datasets \n \nWe a...,110,False
3,1418.0,True,9564100,\n## Materials and Methods \n \n### Participa...,1418,True
4,23.0,True,9548384,\n## Methods \n \n### Participants \n \nThis...,23,True
...,...,...,...,...,...,...
64,56.0,True,5068584,\n## Materials and methods \n \n### Participa...,56,True
65,27.0,True,5903407,\n## Materials and methods \n \n### Participa...,27,True
66,13.0,True,2494616,\n## Methods \n \n### Participants \n \nAll ...,13,True
67,13.0,True,6989437,\n## Methods \n \n### Participants \n \nFift...,13,True


In [23]:
clean_single_preds = clean_predictions(single_preds)

KeyError: 'group_name'

In [ ]:
ix_corr_n_groups_single, ix_more_groups_single, _= evaluate_predictions(single_preds, sub_single_group)

In [292]:
# With "cleaned up" predictions
_= evaluate_predictions(clean_single_preds, sub_single_group)

Exact match # of groups: 0.88
 More groups predicted: 0.12
 Less groups predicted: 0.00

{'age maximum': 0.83,
 'age mean': 0.72,
 'age median': 0.87,
 'age minimum': 0.83,
 'count': 0.74,
 'diagnosis': 0.68,
 'female count': 0.64,
 'group_name': 0.78,
 'male count': 0.67,
 'subgroup_name': 0.0}


### Error types:

- Many times multiple rows get created for a single group study, each time a new "revised" sample size is mention (i.e. after exclusion)
- 2x the human annotation had less detai (i.e. total N instead sub group chars, that were combined into 1 group)

### Compare labels predicted vs given, on subset w/ matching group numbers

In [296]:
subset_pred_corrgroups[subset_pred_corrgroups.pmcid == 9435010]

,count,diagnosis,group_name,subgroup_name,male count,female count,age mean,age range,age minimum,age maximum,age median,pmcid
43,100.0,FES,patients,drugnave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9435010
44,50.0,NaN,healthy,controls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9435010


In [297]:
subset_annot_corrgroups[subset_annot_corrgroups.pmcid == 9435010]

,count,diagnosis,group_name,subgroup_name,male count,female count,age mean,age minimum,age maximum,age median,pmcid
124,100,schizophrenia,patients,_,NaN,NaN,NaN,NaN,NaN,NaN,9435010
125,50,NaN,healthy,_,NaN,NaN,NaN,NaN,NaN,NaN,9435010


In [123]:
# Subset to only look at studies w/ same # of groups
subset_pred_corrgroups = predictions[predictions.pmcid.isin(ix_corr_n_groups)]
subset_annot_corrgroups = sub_multi_group[sub_multi_group.pmcid.isin(ix_corr_n_groups)]

In [126]:
annot_diag = subset_annot_corrgroups[(subset_annot_corrgroups.group_name == 'patients')][['count', 'pmcid', 'diagnosis']]
pred_diag = subset_pred_corrgroups[subset_pred_corrgroups.group_name == 'patients'][['count', 'pmcid', 'diagnosis']]
pd.merge(annot_diag, pred_diag, on=['pmcid', 'count'])

,count,pmcid,diagnosis_x,diagnosis_y
0,15,2648877,Autism Spectrum Disorder,Autism Spectrum Disorder
1,10,3742334,chronic marijuana use,chronic marijuana (MJ) users
2,17,3877773,Tourette syndrome,TS
3,16,3984441,dementia with Lewy bodies,DLB
4,30,4190683,Attention Deficit Hyperactivity Disorder,ADHD
5,18,4265725,Autism spectrum conditions,ASC
6,32,4473263,chronic left-hemisphere stroke,chronic left-hemisphere stroke
7,11,4589842,Depression,depressed patients
8,42,5394595,bronchial asthma without acute attacks,bronchial asthma
9,34,5413198,Alcohol dependence,ADP


In [127]:
annot_diag = subset_annot_corrgroups[(subset_annot_corrgroups.group_name == 'healthy')][['count', 'pmcid', 'diagnosis']]
pred_diag = subset_pred_corrgroups[subset_pred_corrgroups.group_name == 'healthy'][['count', 'pmcid', 'diagnosis']]
pd.merge(annot_diag, pred_diag, on=['pmcid', 'count'])

,count,pmcid,diagnosis_x,diagnosis_y
0,19,2561002,NaN,NaN
1,15,2561002,NaN,NaN
2,18,2648877,NaN,non-autistic control
3,24,3502502,NaN,healthy
4,44,3511796,NaN,healthy
5,18,3742334,NaN,nonusing (NU) comparison subjects
6,15,3877773,NaN,HC
7,17,3984441,NaN,controls
8,11,4589842,NaN,healthy volunteers
9,35,5292583,NaN,NaN


### Other groups

In [182]:
subset_pred_corrgroups[subset_pred_corrgroups.group_name.isin(['healthy', 'patients']) == False][['count', 'pmcid', 'diagnosis', 'group_name']]


,count,pmcid,diagnosis,group_name
12,23.0,8933759,healthy adults,controls
48,20.0,7485713,NaN,controls
53,20.0,6820536,fibromyalgia,FM
54,20.0,6820536,NaN,HC
62,24.0,3965851,NaN,participants
63,18.0,3965851,NaN,participants
81,260.0,3991323,NaN,adolescent
82,28.0,3991323,NaN,adult
83,164.0,4174863,NaN,incarcerated
84,46.0,4174863,NaN,non-incarcerated


### Error types
- Will often fail to set name to `healthy` for controls.
   - Can probably come up with a whitelist of terms that can be translated to `healthy`:
      - ['control', 'volunteers', 'subjects', 'participants', 'adults'] + if diagnosis == `n/a` 

## Look at example of failures

### When model predicted more groups

In [145]:
sub_multi_group[sub_multi_group.pmcid.isin(ix_more_groups)].sort_values('pmcid')

,count,diagnosis,group_name,subgroup_name,male count,female count,age mean,age minimum,age maximum,age median,pmcid
130,16,autism spectrum disorder,patients,_,16.0,NaN,NaN,8.0,18.0,NaN,5279905
131,16,NaN,healthy,_,NaN,NaN,NaN,NaN,NaN,NaN,5279905
136,19,unipolar major depression,patients,validation study,NaN,NaN,NaN,18.0,65.0,NaN,5416685
135,20,unipolar major depression,patients,primary study,NaN,NaN,NaN,18.0,65.0,NaN,5416685
138,19,NaN,healthy,validation study,NaN,NaN,NaN,18.0,65.0,NaN,5416685
137,19,NaN,healthy,primary study,NaN,NaN,NaN,18.0,65.0,NaN,5416685
90,15,NaN,healthy,_,NaN,NaN,NaN,60.0,70.0,NaN,5665859
89,72,ischemic stroke,patients,_,NaN,NaN,NaN,60.0,70.0,NaN,5665859
20,39,NaN,healthy,tsd,22.0,17.0,33.5,NaN,NaN,NaN,7260173
21,15,NaN,healthy,control,7.0,8.0,34.5,NaN,NaN,NaN,7260173


In [146]:
predictions[predictions.pmcid.isin(ix_more_groups)].sort_values('pmcid')

,count,diagnosis,group_name,subgroup_name,male count,female count,age mean,age range,age minimum,age maximum,age median,pmcid
132,16.0,TD,healthy,,16.0,NaN,NaN,NaN,NaN,NaN,NaN,5279905
131,16.0,ASD,patients,,16.0,NaN,13.0,8-18,8.0,18.0,13.0,5279905
130,30.0,healthy,healthy,children,15.0,15.0,9.0,6-13,6.0,13.0,9.0,5279905
129,20.0,ASD,patients,children,10.0,10.0,8.0,5-12,5.0,12.0,8.0,5279905
22,20.0,NaN,healthy,validation sample,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5416685
18,NaN,,,,NaN,NaN,NaN,,NaN,NaN,NaN,5416685
19,20.0,unipolar major depression,patients,primary study,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5416685
20,19.0,unipolar major depression,patients,validation study,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5416685
21,19.0,NaN,healthy,primary sample,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5416685
74,15.0,healthy subjects,healthy,,NaN,NaN,NaN,,NaN,NaN,NaN,5665859


In [164]:
sections[sections.pmcid == 9230060].iloc[0].content

'\n## 2. Materials and Methods \n  \nThis prospective study was approved by the local institutional review board of Kaohsiung Veterans General Hospital (protocol number: VGHKS93-CT2-09). This study enrolled 20 right-handed male offenders who were further separated into 3 sub-groups by their records of court verdict: six offenders had committed affective violence (VA group), seven offenders had committed predatory violence (VP group), and seven had committed non-violent crime (NV group). The VA offenders were defined as the subjects who impulsively or affectively committed violent crime. The VP offenders were defined as the subjects who purposely planned to commit violent crime with detailed documentation in the court records. The NV offenders were defined as the subjects who committed non-violent crime based on the documents in court record. In addition, 20 age-matched right-handed male non-criminal healthy controls (HC group) were enrolled for comparisons. All subjects underwent psych

### Error types
- Sometimes there is repeated information (for example, in Methods and Results). Not clear which section to "trust". In addition there are confusions about total N. 
- It seems to create a group if given the introduction, but it oftne lacks count. Which means we can probably drop those observations
- Final N and exclusion criteria always seems to confuse the models. In the case of multigroup prompt, instead of choosing one, it often will invent another group -- how to handle conflicting information?
- Also: it OFTEN, invents group_names other than `patients` or `healthy`. I think we can assume that if `diagnosis == NaN` or `healthy` then group_name can be corrected to `healthy`, and if there is a diagnosis then group_name should be patient. However, there will be rare times when something meaning healthy is given to diagnosis, and group name is something like 'control'.


Ideas for finding correction section:
- If can find "Methods" in body--- use that and search within that, or sections that follow.
- Also, should save subsection prior to splitting up-- save all headers to be able to select methods section
- If not, then search over entire document.
- Iterate until some information is generated, and iterate if there's no hits


### When model predicted fewer groups

In [147]:
sub_multi_group[sub_multi_group.pmcid.isin(ix_less_groups)].sort_values('pmcid')

,count,diagnosis,group_name,subgroup_name,male count,female count,age mean,age minimum,age maximum,age median,pmcid
57,20,NaN,healthy,dataset 1,NaN,NaN,NaN,NaN,NaN,NaN,4352055
58,19,NaN,healthy,dataset 2,NaN,NaN,NaN,NaN,NaN,NaN,4352055
205,19,NaN,healthy,work with children,9.0,10.0,NaN,NaN,NaN,NaN,7395771
206,19,NaN,healthy,no work with children,9.0,10.0,NaN,NaN,NaN,NaN,7395771
101,88,attention/deficit hyperactivity disorder,patients,_,NaN,NaN,NaN,NaN,NaN,NaN,8782893
102,79,NaN,healthy,_,NaN,NaN,NaN,NaN,NaN,NaN,8782893
163,573,NaN,healthy,abide 1,NaN,NaN,NaN,NaN,NaN,NaN,9407088
164,593,NaN,healthy,abide 2,NaN,NaN,NaN,NaN,NaN,NaN,9407088
161,539,autism spectrum disorder,patients,abide 1,NaN,NaN,NaN,NaN,NaN,NaN,9407088
162,521,autism spectrum disorder,patients,abide 2,NaN,NaN,NaN,NaN,NaN,NaN,9407088


In [148]:
predictions[predictions.pmcid.isin(ix_less_groups)].sort_values('pmcid')

,count,diagnosis,group_name,subgroup_name,male count,female count,age mean,age range,age minimum,age maximum,age median,pmcid
122,20.0,NaN,participants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4352055
64,38.0,healthy,adults,childless,19.0,19.0,24.08,3.33,19.0,29.0,24.0,7395771
9,167.0,ADHD,children and adolescents,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8782893
3,539.0,ASD patients,patients,ASD patients,300.0,239.0,25.00,18-40,18.0,40.0,26.0,9407088
4,573.0,normal controls,healthy,normal controls,300.0,273.0,28.00,20-45,20.0,45.0,29.0,9407088


In [173]:
sections[sections.pmcid == 9407088].iloc[0].content

'\n## 3. Experiments \n  \n### 3.1. Data \n  \nThe data used in this article come from the Autism Brain Imaging Data Exchange (ABIDE) project [ ]. This project aims to accelerate the understanding of the deep brain mechanism of autism spectrum disorder (ASD), which integrates the brain structure and functional imaging data from many laboratories around the world. The abide I and abide II datasets were used. The abide I data were collected from 17 centers, including 1112 subjects, including 539 ASD patients and 573 normal controls; the abide II dataset was collected from 19 centers, including 1114 subjects, including 521 ASD patients and 593 normal controls. \n\nTo construct a 3D custom fMRI template using fMRI time-series data, we first need to preprocess the data. The basic idea of fMRI data preprocessing is to eliminate the timing error of the interlayer scanning and the head movement error caused by the subject’s head movement in the scanning process, and then carry out the time-lay

### Error types
- Combining two groups into 1 (not necessarily wrong)-- happened 2x, usually combining control group w/ patient group
- Missed a dataset (abide)

Overall, not much we can do about this